<a href="https://colab.research.google.com/github/Mrprey/UFAL/blob/master/AS6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

O DataSet é sobre imagens de raio-x para detecção de pneumunia
com mais de 50000 imagens para treino e mais de 500 para o teste

In [93]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [94]:
import numpy as np
import pandas as pd
import cv2 # opencv
import matplotlib.pyplot as plt
import os

from sklearn.metrics import accuracy_score

from keras import preprocessing
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop

# Lendo as imagens

In [95]:
img = cv2.imread('/content/drive/My Drive/Colab Notebooks/DataSets/chest_xray/test/NORMAL/IM-0001-0001.jpeg')

imagem = cv2.resize(img, (19, 29))

imagem



array([[[ 29,  29,  29],
        [ 27,  27,  27],
        [ 26,  26,  26],
        ...,
        [ 28,  28,  28],
        [ 28,  28,  28],
        [ 32,  32,  32]],

       [[ 30,  30,  30],
        [ 24,  24,  24],
        [ 23,  23,  23],
        ...,
        [ 27,  27,  27],
        [ 27,  27,  27],
        [ 29,  29,  29]],

       [[ 31,  31,  31],
        [ 20,  20,  20],
        [ 14,  14,  14],
        ...,
        [ 48,  48,  48],
        [ 18,  18,  18],
        [ 26,  26,  26]],

       ...,

       [[ 25,  25,  25],
        [ 26,  26,  26],
        [ 22,  22,  22],
        ...,
        [187, 187, 187],
        [ 27,  27,  27],
        [ 32,  32,  32]],

       [[ 26,  26,  26],
        [ 28,  28,  28],
        [ 15,  15,  15],
        ...,
        [181, 181, 181],
        [ 27,  27,  27],
        [ 34,  34,  34]],

       [[ 27,  27,  27],
        [ 26,  26,  26],
        [ 18,  18,  18],
        ...,
        [160, 160, 160],
        [ 27,  27,  27],
        [ 31,  31,  31]]

In [96]:
input_dir = '/content/drive/My Drive/Colab Notebooks/DataSets/chest_xray/'

#tipo = train ou test
classes = {'NORMAL':0,'PNEUMONIA':1}
def ler_dataset(tipo):
  X = []
  y = []
  qtd = 5000 if tipo == 'train' else 500
  #para cada pasta
  for pasta in os.listdir(input_dir+tipo+'/'):
    print(pasta)
    #para cada imagem na pasta
    for img_file in os.listdir(input_dir+tipo+'/'+pasta+'/')[:qtd]:
      #leia a imagem
      img = cv2.resize(cv2.imread(input_dir+tipo+'/'+pasta+'/'+img_file), (48, 48))
      #transforme imagem em array de pixels e salve na lista de imagens
      X.append(img)
      #guarde a classificação da imagem
      y.append(classes[pasta])
  X = np.asarray(X)
  y = np.asarray(y)
  return X,y


In [ ]:
X_treino,y_treino = ler_dataset('train')

PNEUMONIA
NORMAL


In [ ]:
X_treino.shape

In [ ]:
y_treino.shape

In [ ]:
X_teste,y_teste = ler_dataset('test')

In [ ]:
X_teste.shape

In [ ]:
#transformando labels de numérico para categórico
y_treino = to_categorical(y_treino, num_classes = 2)
y_teste = to_categorical(y_teste, num_classes = 2)

# Re-escala dos dados

In [ ]:
X_treino = X_treino/255
X_teste = X_teste/255

# Definindo CNN

In [ ]:
model = Sequential()

# 2 camadas de convolução de 32 filtros + Max pooling [2,2]
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (48,48,3)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# 2 camadas de convolução de 64 filtros + Max pooling [2,2]
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.2))

# 2 camadas de convolução de 128 filtros + Max pooling [2,2]
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.2))

#  2 camadas fully conected 
model.add(Flatten())#converte os mapas de ativação finais para um veter de 1 dimensão
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(2, activation = "softmax"))

In [ ]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [ ]:
# Compilando o modelo
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# treinamento do modelo

model.fit(X_treino, y_treino, epochs=1,verbose = 1)